So far, we wrote our code "as we went". But to really step up our game, we need to look into *functions*. It is useful to think of computer functions as mathematic functions.

Let's start with a simple example. We will write a function to simulate tossing a coin -- but this time, the coin can be biased. We want our function to have a few specificities in the way it behaves. First, we will assume that `true` means head. Second, we will assume that the bias measures the extent to which the coin will give tails. An unbiased coin will have a bias value of *0.5*: it has equal chance of resulting in tails or heads.

In [ ]:
function coin_toss(bias::Float64=0.5)
    return rand() < bias
end